In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
!nvidia-smi


Sun Mar  9 14:28:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
tf.debugging.set_log_device_placement(True)


In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
print(f'Found GPU at: {device_name}')


Found GPU at: /device:GPU:0


In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload a file




Saving tokenized_policies.csv to tokenized_policies.csv


In [ ]:
df = pd.read_csv("tokenized_policies.csv")  # Adjust filename if needed
df.head()

,Policy_ID,Normalized_Text,BPE_Tokens
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,personal property coverage includes electronic...,"['▁personal', '▁property', '▁coverage', '▁incl..."
1,23b8c1e9-3924-46de-beb1-3b9046685257,landlord insurance covers rental property and ...,"['▁', 'l', 'and', 'l', 'or', 'd', '▁insurance'..."
2,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,replacement cost coverage pays for rebuilding ...,"['▁repla', 'ce', 'ment', '▁co', 'st', '▁covera..."
3,972a8469-1641-4f82-8b9d-2434e465e150,riders can be added for additional benefits li...,"['▁riders', '▁can', '▁be', '▁added', '▁for', '..."
4,17fc695a-07a0-4a6e-8822-e8f36c031199,hitandrun accidents may be covered under unins...,"['▁hi', 'tand', 'r', 'un', '▁accidents', '▁may..."


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch
import pandas as pd

# Load CSV File
df = pd.read_csv("tokenized_policies.csv")  # Adjust the path if needed

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to Translate in Batches
def batch_translate_texts(texts, tokenizer, model, batch_size=16):
    """Translates a batch of texts using the given tokenizer and model."""
    translations = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)  # Move to GPU

        with torch.no_grad():
            translated_tokens = model.generate(**inputs)

        translations.extend([tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens])
    return translations

# Load Translation Models and Move to GPU
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr").to(device)  # Move model to GPU

tokenizer_es = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model_es = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es").to(device)  # Move model to GPU

# Translate Only the Policy_Text Column
if "Normalized_Text" in df.columns:
    texts = df["Normalized_Text"].fillna("").tolist()  # Replace NaN with empty strings

    # French Translation
    df["French_Translation"] = batch_translate_texts(texts, tokenizer_fr, model_fr, batch_size=16)

    # Spanish Translation
    df["Spanish_Translation"] = batch_translate_texts(texts, tokenizer_es, model_es, batch_size=16)

    # Save Translations Back to CSV
    df.to_csv("translated_policies.csv", index=False, encoding="utf-8")
    print("Translation completed and saved to translated_policies.csv")
else:
    print("Policy_Text column not found in the CSV file")

# Display Preview
df.head()


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translation completed and saved to translated_policies.csv


,Policy_ID,Normalized_Text,BPE_Tokens,French_Translation,Spanish_Translation
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,personal property coverage includes electronic...,"['▁personal', '▁property', '▁coverage', '▁incl...",L'assurance-logement couvre les biens locatifs...,cobertura de bienes personales incluye muebles...
1,23b8c1e9-3924-46de-beb1-3b9046685257,landlord insurance covers rental property and ...,"['▁', 'l', 'and', 'l', 'or', 'd', '▁insurance'...",L'assurance du propriétaire couvre les biens l...,El seguro del propietario cubre la propiedad d...
2,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,replacement cost coverage pays for rebuilding ...,"['▁repla', 'ce', 'ment', '▁co', 'st', '▁covera...",La plupart des stratégies de planification suc...,cobertura de costos de reemplazo paga por la r...
3,972a8469-1641-4f82-8b9d-2434e465e150,riders can be added for additional benefits li...,"['▁riders', '▁can', '▁be', '▁added', '▁for', '...",L'assurance-maladie couvre les frais médicaux ...,Los pasajeros pueden ser añadidos para benefic...
4,17fc695a-07a0-4a6e-8822-e8f36c031199,hitandrun accidents may be covered under unins...,"['▁hi', 'tand', 'r', 'un', '▁accidents', '▁may...",Les accidents de hitandrun peuvent être couver...,Los accidentes de atropello pueden estar cubie...


In [ ]:
df.to_csv("translated_policies.csv", index=False, encoding="utf-8")
print("File saved as translated_policies.csv")


File saved as translated_policies.csv


In [ ]:
from google.colab import files
files.download("translated_policies.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import pipeline
import pandas as pd
import torch
from google.colab import files

# 🔹 Step 1: Manually Upload CSV File
uploaded = files.upload()  # Opens a file selection dialog

# Get the filename (assumes only one file is uploaded)
filename = list(uploaded.keys())[0]
print(f"Uploaded File: {filename}")

# 🔹 Step 2: Load the Dataset
df = pd.read_csv(filename)  # Load the uploaded file

# 🔹 Step 3: Load the Summarization Model (with GPU support)
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

# 🔹 Step 4: Define a Function to Summarize Text Safely
def summarize_text(text):
    if isinstance(text, str) and len(text) > 50:  # Ensure text is long enough to summarize
        return summarizer(text, max_length=25, min_length=10, do_sample=False)[0]['summary_text']
    return text  # Return original if too short or NaN

# 🔹 Step 5: Apply Summarization to the "Normalized_Text" Column
if "Normalized_Text" in df.columns:
    df["Summary"] = df["Normalized_Text"].fillna("").apply(summarize_text)
else:
    print("Error: 'Normalized_Text' column not found in the dataset!")

# 🔹 Step 6: Save the Updated Dataset
df.to_csv("summarized_policies.csv", index=False, encoding="utf-8")
print("Summarization completed and saved as summarized_policies.csv")

# 🔹 Step 7: Provide Option to Download the File
files.download("summarized_policies.csv")

# Display First Few Rows
df.head()


Saving translated_policies.csv to translated_policies (1).csv
Uploaded File: translated_policies (1).csv


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Summarization completed and saved as summarized_policies.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Policy_ID,Normalized_Text,BPE_Tokens,French_Translation,Spanish_Translation,Summary
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,personal property coverage includes electronic...,"['▁personal', '▁property', '▁coverage', '▁incl...",L'assurance-logement couvre les biens locatifs...,cobertura de bienes personales incluye muebles...,Outofnetwork providers may result in higher ou...
1,23b8c1e9-3924-46de-beb1-3b9046685257,landlord insurance covers rental property and ...,"['▁', 'l', 'and', 'l', 'or', 'd', '▁insurance'...",L'assurance du propriétaire couvre les biens l...,El seguro del propietario cubre la propiedad d...,Vision and dental insurance are often separate...
2,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,replacement cost coverage pays for rebuilding ...,"['▁repla', 'ce', 'ment', '▁co', 'st', '▁covera...",La plupart des stratégies de planification suc...,cobertura de costos de reemplazo paga por la r...,Estate planning strategies often include life ...
3,972a8469-1641-4f82-8b9d-2434e465e150,riders can be added for additional benefits li...,"['▁riders', '▁can', '▁be', '▁added', '▁for', '...",L'assurance-maladie couvre les frais médicaux ...,Los pasajeros pueden ser añadidos para benefic...,Highvalue homes may require specialty insuranc...
4,17fc695a-07a0-4a6e-8822-e8f36c031199,hitandrun accidents may be covered under unins...,"['▁hi', 'tand', 'r', 'un', '▁accidents', '▁may...",Les accidents de hitandrun peuvent être couver...,Los accidentes de atropello pueden estar cubie...,Some policies provide coverage for alternative...


parallel corpus step

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a multilingual embedding model
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Compute embeddings for all texts
english_embeddings = embedder.encode(df["Normalized_Text"].fillna("").tolist(), convert_to_tensor=True)
french_embeddings = embedder.encode(df["French_Translation"].fillna("").tolist(), convert_to_tensor=True)
spanish_embeddings = embedder.encode(df["Spanish_Translation"].fillna("").tolist(), convert_to_tensor=True)

# Compute cosine similarities
cosine_sim_fr = util.pytorch_cos_sim(english_embeddings, french_embeddings).diag()
cosine_sim_es = util.pytorch_cos_sim(english_embeddings, spanish_embeddings).diag()

# Add similarity scores to the DataFrame
df["Similarity_EN-FR"] = cosine_sim_fr.cpu().numpy()
df["Similarity_EN-ES"] = cosine_sim_es.cpu().numpy()

# Define the threshold for acceptable similarity
threshold = 0.7

# Compute the percentage of translations above the threshold
fr_similar_count = (df["Similarity_EN-FR"] >= threshold).sum()
es_similar_count = (df["Similarity_EN-ES"] >= threshold).sum()

# Compute total rows
total_rows = len(df)

# Compute percentages
fr_similarity_percent = (fr_similar_count / total_rows) * 100
es_similarity_percent = (es_similar_count / total_rows) * 100

print(f"French Similarity: {fr_similarity_percent:.2f}% of translations have similarity >= {threshold}")
print(f"Spanish Similarity: {es_similarity_percent:.2f}% of translations have similarity >= {threshold}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

French Similarity: 92.60% of translations have similarity >= 0.7
Spanish Similarity: 98.60% of translations have similarity >= 0.7


In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=adf27e32882dad0e136e43bb02e458455e80303a8f9cef5274158e927abb2e14
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
pip install transformers datasets sentencepiece torch evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Seq2SeqTrainer, Seq2SeqTrainingArguments
)
import evaluate
from google.colab import files  # For manual file upload

# 🔹 Step 1: Upload CSV File Manually
uploaded = files.upload()  # Opens a file selection dialog

# Get the uploaded filename
filename = list(uploaded.keys())[0]
print(f"✅ Uploaded File: {filename}")

# Load the dataset
df = pd.read_csv(filename)

# Ensure required columns are present
required_columns = {"Normalized_Text", "Summary"}
missing_columns = required_columns - set(df.columns)
if missing_columns:
    raise ValueError(f"❌ Missing required columns: {missing_columns}")

print(f"✅ Dataset loaded with {len(df)} rows.")

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Normalized_Text', 'Summary']])

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1)
print(f"✅ Dataset split: {len(dataset['train'])} train | {len(dataset['test'])} test")

# 🔹 Step 2: Load Model & Tokenizer (Choose mBART or mT5)
MODEL_NAME = "facebook/mbart-large-50"  # or "google/mt5-small"
print(f"✅ Loading model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# 🔹 Step 3: Preprocessing Function
def preprocess_function(examples):
    inputs = tokenizer(
        examples["Normalized_Text"], padding="longest", truncation=True, max_length=512
    )
    targets = tokenizer(
        examples["Summary"], padding="longest", truncation=True, max_length=128
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["Normalized_Text", "Summary"])
print(f"✅ Tokenization completed!")

# 🔹 Step 4: Define Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=500,
    report_to="none",  # Disable reporting (e.g., WandB, TensorBoard)
)

# 🔹 Step 5: Define Trainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 🔹 Step 6: Train Model
print("🚀 Starting training...")
trainer.train()
print("✅ Training completed!")

# 🔹 Step 7: Save Model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("✅ Fine-tuning completed and model saved to './fine_tuned_model'")

Saving summarized_policies.csv to summarized_policies.csv
✅ Uploaded File: summarized_policies.csv
✅ Dataset loaded with 500 rows.
✅ Dataset split: 450 train | 50 test
✅ Loading model: facebook/mbart-large-50


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

<ipython-input-4-22035557ef01>:87: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


🚀 Starting training...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.337275,51.120000,43.670000,47.100000,47.100000
2,No log,0.278244,54.630000,47.520000,51.500000,51.320000
3,No log,0.280236,55.680000,47.870000,52.410000,52.350000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✅ Training completed!
✅ Fine-tuning completed and model saved to './fine_tuned_model'


In [ ]:
import shutil
import joblib
import pickle
from google.colab import files

# 🔹 Step 7: Save Model
model_dir = "./fine_tuned_model"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# 🔹 Step 8: Save Using Pickle (Optional)
model_path = "fine_tuned_model.sav"
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print(f"✅ Model saved to {model_path}")

# 🔹 Step 9: Zip and Download Model
shutil.make_archive("fine_tuned_model", 'zip', model_dir)  # Create a ZIP file
files.download("fine_tuned_model.zip")  # Prompt file download
print("✅ Model zipped and ready for download!")


✅ Model saved to fine_tuned_model.sav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Model zipped and ready for download!
